# Exploración Inicial de los Datos de RCP Telefónica

Este notebook contiene los primeros pasos para analizar los datos del proyecto de RCP telefónica.

## Importar Librerías Necesarias

Importamos las librerías principales para el análisis de datos y visualización.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de gráficos
%matplotlib inline
sns.set(style="whitegrid")

## Cargar y Visualizar los Datos

Cargamos los archivos de datos desde la carpeta `data` y mostramos las primeras filas de cada uno.

In [ ]:
# Cargar el archivo cleaned_data.csv y mostrar el número de filas
df = pd.read_csv('../data/cleaned_data.csv')
print(f"Número de filas cargadas: {df.shape[0]}")
df.head()